---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

Let us first load the data and see what it looks like, to decide what variables to use and how to proceed here.

Looks like zip codes (and mailing lists etc.) are super specific variabes which could lead to overfitting, and are best left alone. Other things such as the violators name etc. are not something we'd take into account. Also, variables like compliance are very much what we want to predict, so we want to prevent data leakage and exclude them. 

An interesting new variable could be exploring whether the **time taken for from issuance to hearing** influences probability of paying the fine (with the prior belief that the more time has passed, the less people would be bothered).   

In [57]:
import pandas as pd
import numpy as np

def blight_model():
    
    import numpy as np
    import pandas as pd
    import datetime as dt

    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import GridSearchCV
    from sklearn.metrics import roc_auc_score, roc_curve, auc

    train = pd.read_csv('readonly/train.csv', encoding = "ISO-8859-1")
    test = pd.read_csv('readonly/test.csv')
    addresses = pd.read_csv('readonly/addresses.csv')
    latlons = pd.read_csv('readonly/latlons.csv')

    address_latlons = pd.merge(addresses, latlons, on = 'address', how = 'inner')
    train = pd.merge(train, address_latlons, on = 'ticket_id', how = 'inner')
    test = pd.merge(test, address_latlons, on = 'ticket_id', how = 'inner')
    train.set_index('ticket_id', inplace = True)
    test.set_index('ticket_id', inplace = True)

    # train.fillna(method = 'pad', inplace = True)

    train = train[(train['compliance'] == 0) | (train['compliance'] == 1)]
    train['compliance'] = train['compliance'].astype(int)

    train_only_columns = ['payment_amount', 'payment_date', 'payment_status', 
                              'balance_due', 'collection_status', 'compliance']

    model_features = ['agency_name', 'violation_code', 'late_fee', 'fine_amount', 'disposition',
                         'clean_up_cost', 'discount_amount', 'judgment_amount', 'lat', 'lon', 'timespan']

    categorical_features = ['agency_name', 'violation_code', 'disposition']

    # turn the non-numerical features into actual categories
    for i in categorical_features:
        train[i] = train[i].astype('category')
        train[i] = train[i].cat.codes

    # making the time_span category for the training dataset
    train.loc[:, 'ticket_issued_date'] = pd.to_datetime(train.loc[:, 'ticket_issued_date'])
    train.loc[:, 'hearing_date'] = pd.to_datetime(train.loc[:, 'hearing_date'])
    train['timespan'] = train['hearing_date'] - train['ticket_issued_date']
    train['timespan'] = train['timespan'].dt.days
    train.fillna(method = 'pad', inplace = True)

    #same for the testing data
    test.loc[:, 'ticket_issued_date'] = pd.to_datetime(test.loc[:, 'ticket_issued_date'])
    test.loc[:, 'hearing_date'] = pd.to_datetime(test.loc[:, 'hearing_date'])
    test['timespan'] = test['hearing_date'] - test['ticket_issued_date']
    test['timespan'] = test['timespan'].dt.days
    test = test[model_features]
    test.fillna(method = 'pad', inplace = True)

    # turn the non-numerical features into actual categories
    for i in categorical_features:
        test[i] = test[i].astype('category')
        test[i] = test[i].cat.codes

    #split the data in a training (+ validation) and a separate testing dataset
    X = train[model_features]
    y = train['compliance']
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

    from sklearn.ensemble import RandomForestRegressor
    # training the model 
    clf = RandomForestRegressor(n_estimators = 100, max_depth = None, random_state = 0).fit(X_train, y_train)

    # makiing preduction on the training set
    y_score = clf.predict(X_test)
    # building the ROC curve
    FPR, TPR, _ = roc_curve(y_test, y_score)
    clf_auc = auc(FPR, TPR)
    accuracy = clf.score(X_test, y_test)
    print("The area under your ROC curve is {:.4f}".format(clf_auc))
    # making the predictions on the unseen testing set
    test['compliance'] = clf.predict(test[model_features])
    return test['compliance']

In [58]:
blight_model()

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2827: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


The area under your ROC curve is 0.8015


ticket_id
284932    0.2400
285362    0.1400
285361    0.2200
285338    0.5500
285346    0.5700
285345    0.4300
285347    0.5100
285342    0.9500
285530    0.1325
284989    0.1300
285344    0.6300
285343    0.4000
285340    0.1400
285341    0.5100
285349    0.5300
285348    0.3900
284991    0.1400
285532    0.1800
285406    0.1000
285001    0.1500
285006    0.1200
285405    0.1500
285337    0.3300
285496    0.3600
285497    0.2800
285378    0.3300
285589    0.2900
285585    0.2700
285501    0.1100
285581    0.3400
           ...  
376367    0.1200
376366    0.1900
376362    0.0100
376363    0.0700
376365    0.1200
376364    0.1900
376228    0.5400
376265    0.3500
376286    0.7300
376320    0.2600
376314    0.3000
376327    1.0000
376385    0.9800
376435    0.9300
376370    0.9800
376434    0.2400
376459    0.4200
376478    0.0700
376473    0.1100
376484    0.3100
376482    0.1900
376480    0.2100
376479    0.2100
376481    0.2100
376483    0.4600
376496    0.1300
376497    0.1300
3764